In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect

#### STEP 1.1 -> Getting all the URLS

In [216]:
# Needs to be changed to 201 to 301
outfile = open("test.txt", "w")
for i in range(201,301):
    
    base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)

    html = urlopen(base_url)
    soup = BeautifulSoup(html.read(), features="lxml")

    links = []
    regex = "/book/show/."

    for link in soup.find_all("a"):
        if re.match(regex, str(link.get("href"))):
            links.append("https://www.goodreads.com"+link.get("href"))
    
    #issues of duplicates        
    links = list(dict.fromkeys(links))
    
    for link in links:
        outfile.write(link+"\n")
outfile.close()

#### STEP 1.2 -> getting the HTML from the URLs, getting these HTMLs in different files

do not run the code below, this is just to show

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import os

# First, I make sure i am in the directory that has the text file with the URLS
path = "C:/Users/Guillaume/Desktop/ADM_HM3"
os.chdir(path)

i = 1
page = 1

with open("test_urls.txt", "r") as urls:
    # I previously created the 100 directories, from page 201 to page 300
    # I change the working directory for which i want the HTMLs 
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_201"
    os.chdir(path)

    for url in urls.readlines():

        html = urlopen(url)

        soup = BeautifulSoup(html.read(), 'html.parser')
        
        # The logig here is not the best but it works- 
        # Basically, once every 100 urls treated I chance the directory
        
        if (i%101) != 0:
                # We stay in the current directory
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

            i = i + 1

        else :

            page = page + 1
            path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_20"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

This has been done locally

#### STEP 1.3 -> Parsing the HTMLs files to retrieve all the required info

In [21]:
# All functions to retrieve the informations

def get_title(soup):
    title = soup.find(id="bookTitle").contents[0].strip()
    return title

def get_series(soup):
    serie = soup.find(id="bookSeries").get_text().strip()
    return serie

def get_author(soup):
    author = soup.find_all(class_="authorName__container")
    author_list = [x.get_text().strip() for x in author]
    return author_list

def get_pages(soup):
    pages = soup.find(itemprop ="numberOfPages").get_text().split()[0]
    return int(pages)

def get_ratingValue(soup):
    ratingValue = soup.find(itemprop ="ratingValue").get_text().split()[0]
    return float(ratingValue)

def get_ratingCount(soup):
    ratingCount = soup.find(itemprop ="ratingCount").get_text().split()[0].replace(",","")
    return (ratingCount)

def get_reviewCount(soup):
    reviewCount = soup.find(itemprop ="reviewCount").get_text().split()[0].replace(",","")
    return (reviewCount)

def get_date(soup):
    regex = "by.*"
    date = soup.find_all(class_ = "row")
    date = re.sub(regex,"" ,str(date[1].contents[0]).replace("Published","")).strip()
    return date

def get_setting(url):
    string = re.compile("/places/.")
    test = soup.find_all(href=string)
    
    char_list= []
    for a in test:
        char_list.append(str(a.contents[0]))

    return char_list

def get_plot(soup):
    plot = soup.find(id="description").get_text().strip()
    return plot 

def get_char(soup):
    string = re.compile("/characters/.")
    test = soup.find_all(href=string)

    char_list= []
    for a in test:
        char_list.append(str(a.contents[0]))

    return char_list

def get_url(soup):
    return x

In [22]:
# We want 1 TSV file for 1 article
# In order to do this -> we first loop over the HTML
# We check if the language is indeed english
# for each HTML we create a new TSV and input the required DATA

i = 1

while i <= 2:
    html = open("article_"+str(i)+".html", "r", encoding="utf-8")
    soup = BeautifulSoup(html.read(), 'html.parser')
        
    with open("html_tsv_test_"+str(i)+".tsv", 'w', encoding="utf-8") as f_output:
        
        # We check the language of the splot
        if detect(get_plot(soup)) == "en":
            
            test_list_tsv = [get_title(soup), 
                             get_series(soup), 
                             get_author(soup),
                             get_pages(soup),
                             get_ratingValue(soup), 
                             #get_ratingCount(soup), 
                             #get_reviewCount(soup), 
                             get_date(soup),
                             get_setting(soup),
                             get_char(soup), 
                             get_plot(soup)]

            tsv_output = csv.writer(f_output, delimiter='\t')
            tsv_output.writerow(test_list_tsv)
            
            i = i + 1


IndexError: list index out of range

In [20]:
def get_ratingCount(soup):
    ratingCount = soup.find(itemprop ="ratingCount").get_text().split()[0].replace(",","")
    return (ratingCount)

html = open("article_2.html", "r", encoding="utf-8")
soup = BeautifulSoup(html.read(), 'html.parser')

get_ratingCount(soup)

'2527738'

In [ ]:
# Alternative with xpath -> did not find a way to go from soup to tree -> and I don't want to have 2 ways in parallel
    bookTitle = tree.xpath("//h1[@id='bookTitle']/text()")[0].strip()
        bookSeries = tree.xpath('//*[@id="bookSeries"]/a/text()')[0].strip()
        bookAuthors = tree.xpath("//span[@itemprop='name']/text()")[0].strip()
        NumberofPages = int(tree.xpath("//span [@itemprop='numberOfPages']/text()")[0].split()[0])
        ratingValue = float(tree.xpath("//span [@itemprop='ratingValue']/text()")[0].strip())
        ratingCount = int((tree.xpath('//*[@id="bookMeta"]/a[2]/text()')[1].split()[0]).replace(",",""))
        reviewCount = int((tree.xpath("//*[@id='bookMeta']/a[3]/text()")[1].split()[0]).replace(",",""))
        PublishingDate = tree.xpath('//*[@id="details"]/div[2]/text()')[0].split()[1:4]